In [1]:
!pip install pandas
!pip install requests
!pip install lxml

  Using cached numpy-1.26.4-cp310-cp310-win_amd64.whl.metadata (61 kB)
  Using cached pytz-2024.1-py2.py3-none-any.whl.metadata (22 kB)
  Using cached tzdata-2024.1-py2.py3-none-any.whl.metadata (1.4 kB)
   ---------------------------------------- 0.0/11.6 MB ? eta -:--:--
    --------------------------------------- 0.3/11.6 MB 7.9 MB/s eta 0:00:02
   --- ------------------------------------ 1.0/11.6 MB 13.3 MB/s eta 0:00:01
   --- ------------------------------------ 1.0/11.6 MB 13.3 MB/s eta 0:00:01
   --- ------------------------------------ 1.0/11.6 MB 13.3 MB/s eta 0:00:01
   ---- ----------------------------------- 1.4/11.6 MB 6.3 MB/s eta 0:00:02
   ------- -------------------------------- 2.1/11.6 MB 8.0 MB/s eta 0:00:02
   ---------- ----------------------------- 3.0/11.6 MB 9.5 MB/s eta 0:00:01
   ----------- ---------------------------- 3.3/11.6 MB 9.6 MB/s eta 0:00:01
   ------------- -------------------------- 3.9/11.6 MB 9.5 MB/s eta 0:00:01
   --------------- -----------

In [18]:
import pandas as pd
import requests
from lxml import html as lxml_html
import re
import csv

# requests.get 날릴 base_rul
# base_url = f"http://www.law.go.kr/DRF/lawSearch.do?target=prec&OC=hsk9174&type=HTML&display=100&page={}&org=400201&JO=형법"

# 판례 ID받아올 리스트
ID_list = []

# ID가 담긴 하이퍼링크 받아오는 코드 (총 75페이지)

for page in range(1,75):
    url = f"http://www.law.go.kr/DRF/lawSearch.do?target=prec&OC=hsk9174&type=HTML&display=100&page={page}&org=400201&JO=형법"
    response = requests.get(url)

    # html로 받았으므로, html로 파싱
    tree = lxml_html.fromstring(response.content)

    # XPath 주소로 하이퍼링크 주소에 접근하기
    addresses = tree.xpath("/html/body/form/div/table/tbody/tr/td/a/@href")

    # 해당 주소가 ID_number라는 리스트로 담김. 
    # 해당 리스트 요소에서 정규표현식을 통해 숫자 제외 전부 제외
    for address in addresses:
        match = re.search(r'ID=(\d+)', address)
        ID = match.group(1)
        ID_list.append(ID)

# 생성된 ID_list에 관하여 CSV파일로 바로 저장
with open('ID_list.csv', 'w', newline ='') as file:
    writer = csv.writer(file)
    writer.writerow(ID_list)

In [19]:
import csv

# 리스트를 초기화합니다
data = []

# CSV 파일을 엽니다
with open('ID_list.csv', 'r') as file:
    reader = csv.reader(file)
    for row in reader:
        data.extend(row)

print(data)

['239459', '239467', '238463', '239247', '239229', '239199', '239237', '239231', '239243', '239227', '238861', '238867', '239263', '238907', '238931', '238933', '238027', '238025', '238021', '237915', '237941', '237943', '237865', '237547', '237597', '237581', '237217', '238945', '238949', '238643', '238637', '238961', '238979', '236821', '238991', '236931', '238999', '235563', '235565', '235557', '235555', '239005', '235553', '235559', '235561', '239003', '238997', '239009', '235107', '239011', '239013', '238405', '234985', '234803', '234935', '238409', '238411', '238413', '238421', '234709', '234711', '234707', '234713', '234701', '234715', '234599', '234443', '234453', '238221', '234401', '234321', '238227', '234335', '233983', '233981', '238231', '233887', '238235', '233889', '238233', '233885', '238239', '233585', '238249', '233587', '238247', '238253', '238251', '233583', '233581', '238241', '238263', '233091', '234057', '233045', '233093', '233095', '233049', '233053', '232763',

In [20]:
len(data)

7345

In [58]:
base_url = "https://www.law.go.kr/LSW/precInfoP.do?precSeq="

for ID in data[:2]:
    
    url = base_url + str(ID)
    
    response = requests.get(url)

    tree = lxml_html.fromstring(response.content)

    # 판례 ID:
    case_ids = ID

    # 판시사항: 
    case_details = tree.xpath('//*[@id="conScroll"]/p[1]')

    # 판결요지: 
    judgment_summaries = tree.xpath('//*[@id="conScroll"]/p[2]')

    # 참조조문: 
    referenced_statutes = tree.xpath('//*[@id="conScroll"]/p[3]/a')

    # 참조판례: 
    referenced_precedents = tree.xpath('//*[@id="conScroll"]/p[4]')

    # 변호인: 
    attorneys = tree.xpath('//*[@id="conScroll"]/p[7]')

    # 주문: 
    orders = tree.xpath('//*[@id="conScroll"]/p[9]')

    # 이유: 
    reasons = tree.xpath('//*[@id="conScroll"]/p[10]')
    
    # 대법관: 
    judges = tree.xpath('//*[@id="conScroll"]/div')

    
# Create a DataFrame from the lists
df = pd.DataFrame({
    'Case ID': [case_ids],
    'Case Details': [case_details],
    'Judgment Summary': [judgment_summaries],
    'Referenced Statutes': referenced_statutes,
    'Referenced Precedents': referenced_precedents,
    'Attorney': attorneys,
    'Order': orders,
    'Reason': reasons,
    'Judge': judges
})
df

ValueError: All arrays must be of the same length

In [56]:
case_details

[' [1] 주거침입죄의 구성요건적 행위인 ‘침입’의 의미 및 침입에 해당하는지 판단하는 기준 / 외부인의 출입이 엄격히 통제되는 건조물에 거주자나 관리자의 승낙 없이 몰래 들어간 경우 또는 출입 당시 거주자나 관리자가 출입의 금지나 제한을 하였음에도 이를 무시하고 출입한 경우, 침입행위에 해당할 수 있는지 여부',
 ' [2] 피고인이 ‘甲에게 100m 이내로 접근하지 말 것’ 등을 명하는 법원의 접근금지가처분 결정이 있는 등 피고인이 甲을 방문하는 것을 甲이 싫어하는 것을 알고 있음에도 임의로 甲이 근무하는 사무실 안으로 들어감으로써 건조물에 침입하였다는 공소사실로 기소된 사안에서, 피고인이 위 결정에 반하여 甲이 근무하는 사무실에 출입한 것은 甲의 명시적인 의사에 반하는 행위일 뿐만 아니라, 출입의 금지나 제한을 무시하고 출입한 경우로서 출입 당시 객관적·외형적으로 드러난 행위태양을 기준으로 보더라도 사실상 평온상태가 침해된 것으로 볼 수 있으므로 건조물침입죄가 성립한다고 한 사례']

In [54]:
df = pd.DataFrame({
    'Case ID': case_ids,
    'Case Details': case_details,
    'Judgment Summary': judgment_summaries,
    'Referenced Statutes': referenced_statutes,
    'Referenced Precedents': referenced_precedents,
    'Attorney': attorneys,
    'Order': orders,
    'Reason': reasons,
    'Judge': judges
})
df

ValueError: All arrays must be of the same length

In [69]:
!pip install tqdm

  Using cached tqdm-4.66.2-py3-none-any.whl.metadata (57 kB)
Using cached tqdm-4.66.2-py3-none-any.whl (78 kB)


In [70]:
import pandas as pd
import requests
from lxml import html as lxml_html
from tqdm import tqdm

base_url = "https://www.law.go.kr/LSW/precInfoP.do?precSeq="

# Initialize empty lists for each column
case_ids = []
case_details = []
judgment_summaries = []
referenced_statutes = []
referenced_precedents = []
attorneys = []
orders = []
reasons = []
judges = []

for ID in tqdm(data):
    url = base_url + str(ID)
    response = requests.get(url)
    tree = lxml_html.fromstring(response.content)

    # 요소 추가
    case_ids.append(ID)
    case_details.append(tree.xpath('//*[@id="conScroll"]/p[1]/text()'))
    judgment_summaries.append(tree.xpath('//*[@id="conScroll"]/p[2]/text()'))
    referenced_statutes.append(tree.xpath('//*[@id="conScroll"]/p[3]/a/text()'))
    referenced_precedents.append(tree.xpath('//*[@id="conScroll"]/p[4]/text()'))
    
    orders.append(tree.xpath('//*[@id="conScroll"]/p[@class="pty4_dep1"][last()-1]/text()'))
    reasons.append(tree.xpath('//*[@id="conScroll"]/p[@class="pty4_dep1"][last()]/text()'))
    judges.append(tree.xpath('//*[@id="conScroll"]/div/text()'))
    if '변호사' in tree.xpath('//*[@id="conScroll"]/p[7]/text()')[0]:
        attorneys.append(tree.xpath('//*[@id="conScroll"]/p[7]/text()'))
    else:
        attorneys.append([])

# 딕셔너리로 리스트를 요소로 넣어 데이터프레임 만들기
df = pd.DataFrame({
    'Case ID': case_ids,
    'Case Details': case_details,
    'Judgment Summary': judgment_summaries,
    'Referenced Statutes': referenced_statutes,
    'Referenced Precedents': referenced_precedents,
    'Attorney': attorneys,
    'Order': orders,
    'Reason': reasons,
    'Judge': judges
})

df


  2%|▏         | 137/7345 [01:43<1:31:08,  1.32it/s]


KeyboardInterrupt: 

In [ ]:
df.to_csv('case.csv')

In [72]:
!pip install aiohttp

  Using cached attrs-23.2.0-py3-none-any.whl.metadata (9.5 kB)
  Using cached frozenlist-1.4.1-cp310-cp310-win_amd64.whl.metadata (12 kB)
  Using cached yarl-1.9.4-cp310-cp310-win_amd64.whl.metadata (32 kB)
  Using cached async_timeout-4.0.3-py3-none-any.whl.metadata (4.2 kB)
   ---------------------------------------- 0.0/370.5 kB ? eta -:--:--
   - -------------------------------------- 10.2/370.5 kB ? eta -:--:--
   -------- ------------------------------ 81.9/370.5 kB 907.3 kB/s eta 0:00:01
   ---------------------------------------  368.6/370.5 kB 2.9 MB/s eta 0:00:01
   ---------------------------------------- 370.5/370.5 kB 2.6 MB/s eta 0:00:00
Using cached aiosignal-1.3.1-py3-none-any.whl (7.6 kB)
Using cached async_timeout-4.0.3-py3-none-any.whl (5.7 kB)
Using cached attrs-23.2.0-py3-none-any.whl (60 kB)
Using cached frozenlist-1.4.1-cp310-cp310-win_amd64.whl (50 kB)
Using cached yarl-1.9.4-cp310-cp310-win_amd64.whl (76 kB)


In [95]:
!pip install numpy

In [100]:
import pandas as pd
import asyncio
import aiohttp
from lxml import html as lxml_html
from tqdm import tqdm
import nest_asyncio
import numpy as np

# Apply nest_asyncio to enable nested running of asyncio
nest_asyncio.apply()

base_url = "https://www.law.go.kr/LSW/precInfoP.do?precSeq="

# Initialize empty lists for each column
case_info = []
case_ids = []
case_details = []
judgment_summaries = []
referenced_statutes = []
referenced_precedents = []
attorneys = []
orders = []
reasons = []
judges = []

async def fetch(ID, session):
    url = base_url + str(ID)
    async with session.get(url) as response:
        return await response.text()

async def main():
    async with aiohttp.ClientSession() as session:
        for i in tqdm(range(0, len(data[:50]), 50)):
            tasks = []
            for ID in data[i:i+100]:
                tasks.append(fetch(ID, session))
            htmls = await asyncio.gather(*tasks)
            for ID, html in zip(data[i:i+100], htmls):
                tree = lxml_html.fromstring(html)
                # Append the extracted data to the respective lists
                case_info.append(tree.xpath('//*[@id="contentBody"]/div[1]/text()'))
                case_ids.append(ID)

                # 각 요소의 기본 XPath 위치
                base_positions = [1, 2, 3, 4]

                for base_position in base_positions:
                    # 판시사항
                    if base_position == 1:
                        case_detail = tree.xpath(f'//*[@id="conScroll"]/p[{base_position}]/text()')
                        if case_detail
                            case_details.append(case_detail)
                        else:
                            base_position -= 1
                            case_details.append(np.nan)
                    # 판결요지
                    elif base_position == 2:
                        judgment_summary = tree.xpath(f'//*[@id="conScroll"]/p[{base_position}]/text()')
                        if judgment_summary:
                            judgment_summaries.append(judgment_summary)
                        else:
                            base_position -= 1
                            judgment_summaries.append(np.nan)
                    # 참조조문
                    elif base_position == 3:
                        referenced_statute = tree.xpath(f'//*[@id="conScroll"]/p[{base_position}]/a/text()')
                        if referenced_statute:
                            referenced_statutes.append(referenced_statute)
                        else:
                            base_position -= 1
                            referenced_statutes.append(np.nan)
                    # 참조판례
                    elif base_position == 4:
                        reference_text = tree.xpath(f'//*[@id="conScroll"]/p[{base_position}]//text()')
                        if reference_text and '판결' in ''.join(reference_text):
                            referenced_precedents.append(reference_text)
                        else:
                            referenced_precedents.append(np.nan)


                orders.append(tree.xpath('//*[@id="conScroll"]/p[@class="pty4_dep1"][last()-1]/text()'))
                reasons.append(tree.xpath('//*[@id="conScroll"]/p[@class="pty4_dep1"][last()]/text()'))
                judges.append(tree.xpath('//*[@id="conScroll"]/div/text()'))
                attorney_text = tree.xpath('//*[@id="conScroll"]/p[7]/text()')
                if attorney_text and '변호사' in ''.join(attorney_text):
                    attorneys.append(attorney_text)
                else:
                    attorneys.append(np.nan)
            # Wait for 3 seconds
            await asyncio.sleep(5)

# Run the main function
await main()

# Create a DataFrame from the lists
df = pd.DataFrame({
    'Case Info' : case_info,
    'Case ID': case_ids,
    'Case Details': case_details,
    'Judgment Summary': judgment_summaries,
    'Referenced Statutes': referenced_statutes,
    'Referenced Precedents': referenced_precedents,
    'Attorney': attorneys,
    'Order': orders,
    'Reason': reasons,
    'Judge': judges
})

df


100%|██████████| 1/1 [00:07<00:00,  7.52s/it]


,Case Info,Case ID,Case Details,Judgment Summary,Referenced Statutes,Referenced Precedents,Attorney,Order,Reason,Judge
0,[\n [대법원 2024. 2. 15. 선고 2023도15164 판결]...,239459,"[ [1] 공동주택 내부의 엘리베이터, 공용 계단, 복도 등 공용 부분이 주거침입죄...",[ [1] 다가구용 단독주택이나 다세대주택·연립주택·아파트와 같은 공동주택 내부의 ...,"[형법 제319조 제1항, 형법 제319조 제1항, 형법 제319조 제1항]","[ [1] , 대법원 2009. 9. 10. 선고 2009도4335 판결, (공20...",NaN,"[ 원심판결을 파기하고, 사건을 서울북부지방법원으로 환송한다.]","[ 상고이유를 판단한다. , 1. 이 사건 공소사실의 요지, 피고인은, ① 2...",[\n 대법관 이동원(재판장) 김상환 권영준(주심) \n ]
1,[\n [대법원 2024. 2. 8. 선고 2023도16595 판결]\...,239467,[ [1] 주거침입죄의 구성요건적 행위인 ‘침입’의 의미 및 침입에 해당하는지 판단...,[ [1] 주거침입죄는 사실상 주거의 평온을 보호법익으로 한다. 주거침입죄의 구성요...,"[형법 제319조 제1항, 형법 제319조 제1항]","[ [1] , 대법원 2022. 3. 24. 선고 2017도18272 전원합의체 판...",[ 변호사 장원철],"[ 원심판결 중 2021. 9. 7. 건조물침입 부분을 파기하고, 이 부분 사건을 ...","[ 상고이유를 판단한다. , 1. 2021. 9. 7. 건조물침입 부분 공소사실...",[\n 대법관 노태악(재판장) 김선수 오경미 서경환(주심) \n ...
2,[\n [대법원 2024. 1. 25. 선고 2020도10330 판결]...,238463,[ 소송사기죄 적용의 엄격성 및 소송사기를 유죄로 인정할 수 있는 경우 / 소송당사...,[ 소송사기는 법원을 속여 자기에게 유리한 판결을 얻음으로써 상대방의 재물 또는 재...,[형법 제347조 제1항],"[대법원 2004. 6. 25. 선고 2003도7124 판결, (공2004하, 12...",[ 법무법인 청률 담당변호사 김태창],"[ 원심판결을 파기하고, 사건을 서울동부지방법원에 환송한다.]","[ 상고이유를 판단한다. , 1. 이 사건 공소사실의 요지, 공소외인은 피고인 ...",[\n 대법관 서경환(재판장) 김선수 노태악(주심) 오경미 \n ...
3,[\n [대법원 2024. 1. 4. 선고 2021도5723 판결]\n...,239247,[에서 몰수 대상으로 정한 ‘범죄행위에 제공한 물건’의 의미 / 위 조항에 의한 몰...,[의 ‘범죄행위에 제공한 물건’은 범죄의 실행행위 자체에 사용한 물건만 의미하는 것...,[구 형법(2020. 12. 8. 법률 제17571호로 개정되기 전의 것) 제48조...,"[대법원 2006. 9. 14. 선고 2006도4075 판결, (공2006하, 17...",[ 변호사 강준성],"[ 원심판결을 파기하고, 사건을 수원지방법원에 환송한다.]","[ 상고이유를 판단한다., (, 등 참조). , (대법원 2008. 4. 24. ...",[\n 대법관 권영준(재판장) 이동원 천대엽(주심) \n ]
4,[\n [대법원 2024. 1. 4. 선고 2022도15955 판결]\...,239229,[ 주거침입죄의 구성요건적 행위인 ‘침입’의 의미 및 침입행위에 해당하는지 판단하는...,[ 주거침입죄는 사실상 주거의 평온을 보호법익으로 한다. 주거침입죄의 구성요건적 행...,[형법 제319조 제1항],"[대법원 2022. 3. 24. 선고 2017도18272 전원합의체 판결, (공20...",[ 법무법인(유한) 클라스한결 담당변호사 조용현 외 5인],[ 원심판결 중 피고인 1·피고인 3·피고인 4·피고인 5·피고인 6에 대한 유죄 ...,"[ 상고이유를 판단한다. , 1. 검사의 상고이유에 대한 판단, 원심은 판시와...",[\n 대법관 권영준(재판장) 이동원 천대엽(주심) \n ]
...,...,...,...,...,...,...,...,...,...,...
95,[\n [대법원 2022. 12. 29. 선고 2022도10660 판결...,233093,[ [1] 신설된 포괄일죄 처벌법규가 시행되기 이전의 행위에 대하여 신설된 법규를 ...,[ [1] 포괄일죄에 관한 기존 처벌법규에 대하여 그 표현이나 형량과 관련한 개정을...,"[형법 제1조 제1항, 형사소송법 제298조 제1항]","[ [1] , 대법원 2016. 1. 28. 선고 2015도15669 판결, / ...",[ 변호사 김정환 외 1인],"[ 원심판결을 파기하고, 사건을 수원고등법원에 환송한다.]","[ 상고이유를 판단한다. , 1. 이 사건 공소사실 중 「아동·청소년의 성보호에...",[\n 대법관 노태악(재판장) 박정화(주심) 김선수 오경미 \n ...
96,[\n [대법원 2022. 12. 29. 선고 2022도12494 판결...,233095,[ [1] 형법상 ‘절취’의 의미 및 사기죄에서 처분행위가 갖는 역할과 기능 / 피...,[ [1] 형법상 절취란 타인이 점유하고 있는 자기 이외의 자의 소유물을 점유자의 ...,"[형법 제329조, 제347조, 형법 제329조, 제347조 제1항]","[ [1] , 대법원 1991. 1. 11. 선고 90도2180 판결, (공1991...",[ 변호사 박규철],[ 상고를 기각한다.],"[ 상고이유를 판단한다. , 1. 형법상 절취란 타인이 점유하고 있는 자기 이외...",[\n 대법관 안철상(재판장) 노정희 이흥구(주심) 오석준 \n ...
97,[\n [대법원 2022. 12. 22. 선고 2020도16420 전원...,233049,[ [1] 범죄의 성립과 처벌에 관하여 규정한 형벌법규 자체 또는 그로부터 수권 내...,[ [1] [다수의견] 범죄 후 법률이 변경되어 그 행위가 범죄를 구성하지 아니하게...,"[형법 제1조 제2항, 형사소송법 제326조 제4호, 형법 제1조 제2항, 구 도로...","[ [1] , 대법원 1963. 1. 31. 선고 62도257 판결(집11-1, 형...",[ 변호사 조예경],"[ 원심판결을 파기하고, 사건을 서울서부지방법원에 환송한다.]","[ 상고이유를 판단한다. , 1. 사건의 개요와 쟁점 , 가. 이 사건 공소...",[\n 대법원장 김명수(재판장) 조재연 박정화 안철상 민유숙 김선수 ...
98,[\n [대법원 2022. 12. 22. 선고 2020도8682 전원합...,233053,[ 배임죄의 주체인 ‘타인의 사무를 처리하는 자’의 의미 / 금전채권채무 관계에서 ...,[ 배임죄는 타인의 사무를 처리하는 자가 임무에 위배하는 행위로써 재산상의 이익을 ...,[형법 제355조 제2항],"[대법원 1989. 7. 25. 선고 89도350 판결(공1989, 1317)(변경...",NaN,"[ 원심판결을 파기하고, 사건을 서울북부지방법원에 환송한다.]","[ 직권으로 판단한다. , 1. 가. 배임죄는 타인의 사무를 처리하는 자가 임무...",[\n 대법원장 김명수(재판장) 조재연 박정화 안철상 민유숙 김선수 ...


In [101]:
df.isnull().sum()

Case Info                 0
Case ID                   0
Case Details              0
Judgment Summary          4
Referenced Statutes       7
Referenced Precedents    40
Attorney                 58
Order                     0
Reason                    0
Judge                     0
dtype: int64

In [98]:
missing_referenced_precedents = df[df['Referenced Precedents'].isna()]['Case ID']
missing_referenced_precedents

11    238867
18    238021
19    237915
27    238945
28    238949
31    238961
32    238979
33    236821
34    238991
36    238999
38    235565
41    239005
44    235561
45    239003
46    238997
47    239009
48    235107
49    239011
50    239013
51    238405
55    238409
56    238411
57    238413
58    238421
63    234701
65    234599
68    238221
70    234321
71    238227
75    238231
77    238235
79    238233
80    233885
81    238239
83    238249
85    238247
86    238253
87    238251
90    238241
91    238263
Name: Case ID, dtype: object

In [77]:
df.to_csv('case.csv', index=False)

In [93]:
df['Referenced Statutes'][0]

[' [1] ',
 '형법 제319조 제1항',
 ' ',
 '[2] ',
 '형법 제319조 제1항',
 ' ',
 '[3] ',
 '형법 제319조 제1항']

In [ ]:
response = requests.get('')
//*[@id="conScroll"]